In [1]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 21.4 MB/s eta 0:00:00


In [1]:
!pip install opencv-python deepface numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=71c484c47c91cbf351e9302f62d1d945c1994509ee27fd153af8b270af2ae63c
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [2]:
from transformers import AutoImageProcessor, AutoModelForVideoClassification
from google.colab import files
from deepface import DeepFace
import numpy as np
import cv2

25-02-17 09:11:30 - Directory /root/.deepface has been created
25-02-17 09:11:30 - Directory /root/.deepface/weights has been created


In [3]:
# Load model directly
processor = AutoImageProcessor.from_pretrained("gautamtata/videomae-base-finetuned-kinetics-finetuned-lipsync-subset-1")
model = AutoModelForVideoClassification.from_pretrained("gautamtata/videomae-base-finetuned-kinetics-finetuned-lipsync-subset-1")

preprocessor_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/944 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/345M [00:00<?, ?B/s]

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("video-classification", model="gautamtata/videomae-base-finetuned-kinetics-finetuned-lipsync-subset-1")

Device set to use cuda:0


In [ ]:
# prompt: print path to my pretrained model

model.config._name_or_path

'gautamtata/videomae-base-finetuned-kinetics-finetuned-lipsync-subset-1'

In [5]:
def detect_motion_and_emotions(video_path, output_path='output_with_motion_emotion.avi'):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Initialize video writer to save output
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width * 2, frame_height))


    ret, prev_frame = cap.read()# Variables to track motion
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if ret else None

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Copy the original frame for visualization
        original_frame = frame.copy()


        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)# Convert current frame to grayscale

        # Detect motion by comparing frames
        if prev_gray is not None:
            diff_frame = cv2.absdiff(prev_gray, gray)
            _, thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw motion contours
            for contour in contours:
                if cv2.contourArea(contour) > 500:  # Ignore small movements
                    (x, y, w, h) = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


        try:# Detect emotions in the frame
            # Analyze emotions using DeepFace
            results = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            dominant_emotion = max(results[0]['emotion'], key=results[0]['emotion'].get)

            # Display the detected emotion on the frame
            cv2.putText(frame, f"Emotion: {dominant_emotion}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        except Exception as e:
            cv2.putText(frame, "No face detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Concatenate the original and processed frames side by side
        combined_frame = np.hstack((original_frame, frame))

        # Save the combined frame
        out.write(combined_frame)

        # Display progress
        frame_count += 1
        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames...")

        # Update previous frame
        prev_gray = gray

    # Release resources
    cap.release()
    out.release()
    print(f"Processed video saved to {output_path}")

    # Allow the user to download the final processed video
    print("Preparing the processed video for download...")
    files.download(output_path)
    return output_path



In [ ]:
# Upload and process a video
print("Please upload a video file (e.g., recorded video):")
uploaded = files.upload()

if uploaded:
    video_path = list(uploaded.keys())[0]  # Get uploaded file name
    detect_motion_and_emotions(video_path)


Please upload a video file (e.g., recorded video):


In [ ]:
def detect_motion_and_emotions(video_path, output_path='output_with_motion_emotion.avi'):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Initialize video writer to save output
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    # Variables to track motion
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if ret else None

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect motion by comparing frames
        if prev_gray is not None:
            diff_frame = cv2.absdiff(prev_gray, gray)
            _, thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw motion contours
            for contour in contours:
                if cv2.contourArea(contour) > 500:  # Ignore small movements
                    (x, y, w, h) = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Detect emotions in the frame
        try:
            # Analyze emotions using DeepFace
            results = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            dominant_emotion = max(results[0]['emotion'], key=results[0]['emotion'].get)

            # Display the detected emotion on the frame
            cv2.putText(frame, f"Emotion: {dominant_emotion}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        except Exception as e:
            cv2.putText(frame, "No face detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save the current frame
        out.write(frame)

        # Update previous frame
        prev_gray = gray

        # Display progress
        frame_count += 1
        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames...")

    # Release resources
    cap.release()
    out.release()
    print(f"Processed video saved to {output_path}")


# Upload and process a video
print("Please upload a video file (e.g., recorded video):")
uploaded = files.upload()

if uploaded:
    video_path = list(uploaded.keys())[0]  # Get uploaded file name
    detect_motion_and_emotions(video_path, output_path='output_with_motion_emotion.avi')

    # Allow the user to download the processed video
    from google.colab import files
    files.download('output_with_motion_emotion.avi')


Please upload a video file (e.g., recorded video):


Saving aae62fa2b80643b8bd88bb6ef5dec6d4_yN4SPyKn.m4v to aae62fa2b80643b8bd88bb6ef5dec6d4_yN4SPyKn.m4v
Processed 10 frames...
Processed 20 frames...
Processed 30 frames...
Processed 40 frames...
Processed 50 frames...
Processed 60 frames...
Processed 70 frames...
Processed 80 frames...
Processed 90 frames...
Processed 100 frames...
Processed 110 frames...
Processed 120 frames...
Processed 130 frames...
Processed 140 frames...
Processed 150 frames...
Processed 160 frames...
Processed 170 frames...
Processed 180 frames...
Processed 190 frames...
Processed 200 frames...
Processed 210 frames...
Processed 220 frames...
Processed 230 frames...
Processed 240 frames...
Processed 250 frames...
Processed 260 frames...
Processed 270 frames...
Processed 280 frames...
Processed 290 frames...
Processed 300 frames...
Processed 310 frames...
Processed 320 frames...
Processed 330 frames...
Processed 340 frames...
Processed 350 frames...
Processed 360 frames...
Processed 370 frames...
Processed 380 frame

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def detect_motion_and_emotions(video_path, output_path='output_with_motion_emotion.avi'):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Initialize video writer to save output
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    # Variables to track motion
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if ret else None

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect motion by comparing frames
        if prev_gray is not None:
            diff_frame = cv2.absdiff(prev_gray, gray)
            _, thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw motion contours
            for contour in contours:
                if cv2.contourArea(contour) > 500:  # Ignore small movements
                    (x, y, w, h) = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Detect emotions in the frame
        try:
            # Analyze emotions using DeepFace
            results = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
            dominant_emotion = max(results[0]['emotion'], key=results[0]['emotion'].get)

            # Display the detected emotion on the frame
            cv2.putText(frame, f"Emotion: {dominant_emotion}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        except Exception as e:
            cv2.putText(frame, "No face detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Save the current frame
        out.write(frame)

        # Update previous frame
        prev_gray = gray

        # Display progress
        frame_count += 1
        if frame_count % 10 == 0:
            print(f"Processed {frame_count} frames...")

    # Release resources
    cap.release()
    out.release()
    print(f"Processed video saved to {output_path}")


# Upload and process a video
print("Please upload a video file (e.g., recorded video):")
uploaded = files.upload()

if uploaded:
    video_path = list(uploaded.keys())[0]  # Get uploaded file name
    detect_motion_and_emotions(video_path, output_path='output_with_motion_emotion.avi')

    # Allow the user to download the processed video
    from google.colab import files
    files.download('output_with_motion_emotion.avi')


Please upload a video file (e.g., recorded video):


Saving expression-vedio_GGII6KFL.m4v to expression-vedio_GGII6KFL.m4v
Processed 10 frames...
Processed 20 frames...
Processed 30 frames...
Processed 40 frames...
Processed 50 frames...
Processed 60 frames...
Processed 70 frames...
Processed 80 frames...
Processed 90 frames...
Processed 100 frames...
Processed 110 frames...
Processed 120 frames...
Processed 130 frames...
Processed 140 frames...
Processed 150 frames...
Processed 160 frames...
Processed 170 frames...
Processed 180 frames...
Processed 190 frames...
Processed 200 frames...
Processed 210 frames...
Processed video saved to output_with_motion_emotion.avi


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>